In [1]:
!pip install pydub
!pip install google-cloud-speech
import pandas as pd
import multiprocessing as mp
import os
import requests
import shutil
from pydub import AudioSegment
from google.cloud import storage
from google.cloud import speech_v1p1beta1 as speech

#from google.cloud.speech import enums
#from google.cloud.speech import types

from google.cloud.speech_v1p1beta1 import enums
from google.cloud.speech_v1p1beta1 import types


from random import randint
import time
import wave

     |████████████████████████████████| 88 kB 929 kB/s eta 0:00:01
     |████████████████████████████████| 90 kB 2.3 MB/s eta 0:00:011
     |████████████████████████████████| 100 kB 2.5 MB/s ta 0:00:01
  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
     |████████████████████████████████| 1.3 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 2.8 MB/s eta 0:00:01
  Using cached idna-2.9-py2.py3-none-any.whl (58 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached urllib3-1.25.9-py2.py3-none-any.whl (126 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached cachetools-4.1.0-py3-none-any.whl (10 kB)
  Using cached rsa-4.0-py2.py3-none-any.whl (38 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)


ImportError: cannot import name 'storage' from 'google.cloud' (unknown location)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'google_text_to_speech_key.json'
BUCKET_MP3 = "bucket_france_culture_mp3"
BUCKET_WAV = "bucket_france_culture_wav"
BUCKET_TXT = "bucket_france_culture_txt"

## Load list of MP3

In [0]:
df = pd.read_csv('/content/drive/My Drive/france_culture_repo/all_podcasts.csv')
df['mp3_filename'] =df['data-asset-source'].str.split('\/').str[-1]
df['date_time']=pd.to_datetime(df['data-asset-created-date'],unit='s')
df = df[~df.mp3_filename.isnull()]
df = df.sort_values(by='date_time',ascending=True)
df = df.drop_duplicates(['data-asset-source'])
df.head()

,Unnamed: 0,index,Unnamed: 0.1,aria-label,class,data-asset-created-date,data-asset-source,data-asset-surtitle,data-asset-title,data-btn-type,data-business-reference,data-clip-type,data-concept-id,data-diffusion-id,data-duration,data-expression-id,data-media-id,data-network-change,data-short-id,data-surtitle-link,data-timeshift,data-title-link,data-universe-title,disabled,title,mp3_filename,date_time
1963,1963,448,13,Lecture du replay,replay-button paused aod,1.272071e+09,https://s3-eu-west-1.amazonaws.com/cruiser-pro...,Les Chemins de la philosophie,redif. correspondance n°04,aod,12410,reecoute,2fae969a-3f60-11e5-8314-005056a87c89,NaN,3532.0,89cd36ea-a0d7-102d-9c2c-fcfcfc001444,89cd36ea-a0d7-102d-9c2c-fcfcfc001444,NaN,284cc18a,/emissions/les-chemins-de-la-philosophie,NaN,/emissions/les-nouveaux-chemins-de-la-connaiss...,NaN,NaN,Réécouter redif. correspondance n°04,RF_F0D62E25-23FD-4D2C-BA80-1E977E0C126F_GENE.MP3,2010-04-24 01:07:00
1954,1954,439,4,Lecture du replay,replay-button paused aod,1.273219e+09,https://s3-eu-west-1.amazonaws.com/cruiser-pro...,Les Chemins de la philosophie,Islam 5/5 : Interprétation séculière du Coran,aod,12410,reecoute,2fae969a-3f60-11e5-8314-005056a87c89,NaN,3515.0,38529ebe-a360-102d-b821-00163ed90aa0,38529ebe-a360-102d-b821-00163ed90aa0,NaN,3151ef7f,/emissions/les-chemins-de-la-philosophie,NaN,/emissions/les-nouveaux-chemins-de-la-connaiss...,savoirs,NaN,Réécouter Islam 5/5 : Interprétation séculière...,NOUVEAUX_CHEMINS_DE_LA_CONNAISSANCE_07_05_2010...,2010-05-07 08:00:00
1953,1953,438,3,Lecture du replay,replay-button paused aod,1.294650e+09,https://s3-eu-west-1.amazonaws.com/cruiser-pro...,Les Chemins de la philosophie,Les origines 1/5 : Le destin de l'univers,aod,12410,reecoute,2fae969a-3f60-11e5-8314-005056a87c89,NaN,3538.0,b1c8e108-6bc0-102e-80a0-fcfcfc001444,b1c8e108-6bc0-102e-80a0-fcfcfc001444,NaN,327831fc,/emissions/les-chemins-de-la-philosophie,NaN,/emissions/les-nouveaux-chemins-de-la-connaiss...,savoirs,NaN,Réécouter Les origines 1/5 : Le destin de l'un...,Les_nouveaux_chemins_de_la_connaissance--Franc...,2011-01-10 09:00:00
1952,1952,437,2,Lecture du replay,replay-button paused aod,1.294736e+09,https://s3-eu-west-1.amazonaws.com/cruiser-pro...,Les Chemins de la philosophie,Les Origines 2/5 : Au commencement était le Verbe,aod,12410,reecoute,2fae969a-3f60-11e5-8314-005056a87c89,NaN,3504.0,a5e9da56-6bc8-102e-80a0-fcfcfc001444,a5e9da56-6bc8-102e-80a0-fcfcfc001444,NaN,49f97bbb,/emissions/les-chemins-de-la-philosophie,NaN,/emissions/les-nouveaux-chemins-de-la-connaiss...,savoirs,NaN,Réécouter Les Origines 2/5 : Au commencement é...,les_nouveaux_chemins--11_01--NET_f85ebf8b-bf2b...,2011-01-11 09:00:00
1951,1951,436,1,Lecture du replay,replay-button paused aod,1.294823e+09,https://s3-eu-west-1.amazonaws.com/cruiser-pro...,Les Chemins de la philosophie,Les Origines 3/5 : L'origine de l'univers,aod,12410,reecoute,2fae969a-3f60-11e5-8314-005056a87c89,NaN,3568.0,97f9cf96-6bd7-102e-80a0-fcfcfc001444,97f9cf96-6bd7-102e-80a0-fcfcfc001444,NaN,72abcfc4,/emissions/les-chemins-de-la-philosophie,NaN,/emissions/les-nouveaux-chemins-de-la-connaiss...,savoirs,NaN,Réécouter Les Origines 3/5 : L'origine de l'un...,Les_nouveaux_chemins_de_la_connaissance--Franc...,2011-01-12 09:00:00


## Download mp3 to google storage

In [0]:
def dl_mp3(file_url,file_name):
  r = requests.get(file_url, stream=True)
  if r.status_code == 200:
      with open(file_name, 'wb') as f:
          r.raw.decode_content = True
          shutil.copyfileobj(r.raw, f)
          
def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    list_files =  [f.name for f in bucket.list_blobs()]
    return list_files

          
def upload_blob(bucket_name, filename):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(filename)

    blob.upload_from_filename(filename)
    os.remove(filename)
    
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()
    
    
def get_mp3_backlog(df):
  already_stored_mp3_list = set(list_blobs(BUCKET_MP3))
  print(f'in storage : {len(already_stored_mp3_list)} files')
  print(f'overall mp3 list : {len(set(df.mp3_filename))}')
  mp3_to_store = set(df.mp3_filename)-already_stored_mp3_list
  print(f'still to be downloaded : {len(mp3_to_store)}')
  df_to_store = df[df.mp3_filename.isin(mp3_to_store)]
  return df_to_store
  
def upload_backlog_mp3(df):
  df_backlog = get_mp3_backlog(df)
  
  for index, row in df_backlog.iterrows():
    try:
      dl_mp3(row['data-asset-source'],row['mp3_filename'])
      upload_blob(BUCKET_MP3,row['mp3_filename'])
      time.sleep(1+randint(1,5))
    except Exception as e:
      print(f'error for  {row["mp3_filename"]}')
      print(e)
    

In [0]:
upload_backlog_mp3(df)

in storage : 1898 files
overall mp3 list : 1906
still to be downloaded : 8
error for  ACTUALITE_PHILOSOPHIQUE___COMMENT_PARLER_D_ESTHETIQUE_AUJOURD_HUI__.NET_FC_FF60BEEB-5868-4FE8-B30C-6E18155856FE.MP3
HTTPConnectionPool(host='static.franceculture.fr', port=80): Max retries exceeded with url: /sites/default/files/ACTUALITE_PHILOSOPHIQUE___COMMENT_PARLER_D_ESTHETIQUE_AUJOURD_HUI__.NET_FC_FF60BEEB-5868-4FE8-B30C-6E18155856FE.MP3 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7eff85082828>: Failed to establish a new connection: [Errno 110] Connection timed out',))
error for  LES_NOUVEAUX_CHEMINS_DE_LA_CONNAISSANCE.NET_FC_6AA7909C-9255-444E-A642-E6073D2063B2.MP3
HTTPConnectionPool(host='static.franceculture.fr', port=80): Max retries exceeded with url: /sites/default/files/LES_NOUVEAUX_CHEMINS_DE_LA_CONNAISSANCE.NET_FC_6AA7909C-9255-444E-A642-E6073D2063B2.MP3 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7eff84ff5ba8>: Failed t

## Convert mp3 to wav in google storage

In [0]:


def dl_mp3_to_local(bucketname,mp3_filename):
  storage_client = storage.Client()
  bucket = storage_client.get_bucket(BUCKET_MP3)
  blob = bucket.blob(mp3_filename)
  blob.download_to_filename(mp3_filename)

def mp3_to_wav(mp3_file_name):
    print('converting')
    wav_file_name = mp3_file_name.split('.mp3')[0] + '.wav'
    exists = os.path.isfile(wav_file_name)
    if not exists:
      
      sound = AudioSegment.from_mp3(mp3_file_name)
      print('sound ok')
      sound.export(wav_file_name, format="wav")
    
  
def frame_rate_channel(audio_file_name):
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels
      
def stereo_to_mono(audio_file_name):
    sound = AudioSegment.from_wav(audio_file_name)
    sound = sound.set_channels(1)
    sound.export(audio_file_name, format="wav")
    
    
def get_backlog_wav():
  
  stored_mp3_list = set(list_blobs(BUCKET_MP3))
  stored_wav_list = set(list_blobs(BUCKET_WAV))
  
  filename_mp3 = [os.path.splitext(f)[0] for f in stored_mp3_list]
  filename_wav = [os.path.splitext(f)[0] for f in stored_wav_list]
  
  print(f' mp3 in storage : {len(filename_mp3)} files')
  print(f' wav in storage : {len(filename_wav)} files')
  
  return list(set(filename_mp3)-set(filename_wav))


def upload_backlog_wav(f):  
    dl_mp3_to_local(BUCKET_MP3,f+'.mp3')
    mp3_to_wav(f+'.mp3')
    frame_rate, channels = frame_rate_channel(f+'.wav')
    
    if channels > 1:
        stereo_to_mono(f+'.wav')
    
    upload_blob(BUCKET_WAV, f+'.wav')
    os.remove(f+'.mp3')
    
    
def multiprocess_convert_and_upload_to_raw():
  backlog_wav = get_backlog_wav()
  pool = mp.Pool(processes=4)
  pool.map(upload_backlog_wav, backlog_wav)
    
        
  
  
#multiprocess_convert_and_upload_to_raw()
  

In [0]:
def delete_mp3_in_local_folder():
  for file_name in os.listdir():
    #4
    if file_name.lower().endswith('.mp3'):
        #5
        os.remove(file_name)
        
delete_mp3_in_local_folder()

NameError: ignored

## Transcript to text

In [0]:
def get_backlog_txt():
  
  stored_mp3_list = set(list_blobs(BUCKET_MP3))
  stored_txt_list = set(list_blobs(BUCKET_TXT))
  
  filename_mp3 = [os.path.splitext(f)[0] for f in stored_mp3_list]
  filename_txt = [os.path.splitext(f)[0] for f in stored_txt_list]
  
  print(f' mp3 in storage : {len(filename_mp3)} files')
  print(f' txt in storage : {len(filename_txt)} files')
  
  return list(set(filename_mp3)-set(filename_txt))

def transcript_wav_file(wav_filename):
    frame_rate, channels = frame_rate_channel(wav_filename)
    os.remove(wav_filename)
    gcs_uri = f'gs://{BUCKET_WAV}/{wav_filename}'
    print(gcs_uri)
    transcript = ''
    
    client = speech.SpeechClient()
    audio = types.RecognitionAudio(uri=gcs_uri)

    config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=frame_rate,
    language_code='fr-FR',
    enable_speaker_diarization=True,
    diarization_speaker_count=2)

    # Detects speech in the audio file
    operation = client.long_running_recognize(config, audio)
    response = operation.result(timeout=10000)
    result = response.results[-1]
    words_info = result.alternatives[0].words
    
    tag=1
    speaker=""

    for word_info in words_info:
        if word_info.speaker_tag==tag:
            speaker=speaker+" "+word_info.word
        else:
            transcript += "speaker {}: {}".format(tag,speaker) + '\n'
            tag=word_info.speaker_tag
            speaker=""+word_info.word

    transcript += "speaker {}: {}".format(tag,speaker)
    
    #delete_blob(bucket_name, destination_blob_name)
    return transcript
  
def write_transcripts(transcript_filename,transcript):
    f= open(transcript_filename,"w+")
    f.write(transcript)
    f.close()
    
def transcript_and_store(f):
  upload_backlog_wav(f)
  print('start transcripting..')
  transcript = transcript_wav_file(f+'.wav')
  print('finished transcripting..')
  write_transcripts(f+'.txt',transcript)
  upload_blob(BUCKET_TXT, f+'.txt')
  delete_blob(BUCKET_WAV, f+'.wav')
  
  
def multiprocess_transcript():
  backlog_text = get_backlog_txt()
  pool = mp.Pool(processes=4)
  pool.map(transcript_and_store, backlog_text)
  
  
#multiprocess_transcript()
backlog_text = get_backlog_txt()
transcript_and_store(backlog_text[1])

 mp3 in storage : 893 files
 txt in storage : 0 files
converting
sound ok
start transcripting..
gs://bucket_france_culture_wav/Les_nouveaux_chemins_de_la_connaissance--22_05--NET_1f60b6b3-eba8-42d3-8897-91077f14e889_DF.wav
finished transcripting..
